In [2]:
import xarray
import sys
sys.path.insert(0,'../code/')
import numpy as np
from lme_forecast_general import LME

#### Simulate data

In [3]:
n_loc = 20
n_age = 8
n_sex = 2
n_year = 10
N = n_loc*n_age*n_sex*n_year
dimensions = [n_loc, n_age, n_sex, n_year]
np.random.seed(127)
X = np.insert(np.random.randn(N,2),0,1,axis=1)
gamma_true = 0.1
u_true = np.random.randn(n_loc)*np.sqrt(gamma_true)
v_true = np.random.randn(n_loc*n_age)*np.sqrt(gamma_true)
beta_true = [1., .5, .5]
cov = np.random.randn(n_age)
cov_dim = [1, n_age, 1, 1]
Z = np.tile(np.kron(np.identity(n_age),np.ones((n_sex*n_year,1))),(n_loc,1))
Y_true = X.dot(beta_true) + np.kron(np.identity(n_loc),np.ones((n_year*n_sex*n_age,1))).dot(u_true) \
   + np.kron(np.identity(n_loc*n_age),np.ones((n_sex*n_year,1))).dot(v_true)*(Z.dot(cov))
delta_true = .05
Y = Y_true + np.random.randn(N)*np.sqrt(delta_true)
print(Y.shape)
Y2 = X.dot(beta_true) + np.random.randn(N)*np.sqrt(delta_true)
print(np.linalg.solve(np.transpose(X).dot(X),np.transpose(X).dot(Y)))

(3200,)
[1.10618652 0.5135996  0.50696761]


#### Build Model
$$ y = \beta_0 + X\beta + \pi_l + \text{cov}\pi_{l,a} $$

In [4]:
model = LME(dimensions, 1, Y, [(X[:,1],dimensions), (X[:,2], dimensions), (cov, cov_dim)], 
            [], [0,1], [(2,[n_age,1,1])], global_intercept=True, ran_intercepts=[[1,1,1]])

In [5]:
model.optimize()

n_groups 20
k_beta 3
k_gamma 9
total number of fixed effects variables 13
fit with gamma fixed...
finished...elapsed 0.6485962867736816


In [6]:
model.beta_soln

array([1.12947817, 0.50012956, 0.50678926])

In [7]:
model.gamma_soln

array([0.08051535, 0.0972084 , 0.0972084 , 0.0972084 , 0.0972084 ,
       0.0972084 , 0.0972084 , 0.0972084 , 0.0972084 ])

In [8]:
model.delta_soln

array([0.04971155])